## 1: Database Connection

In [1]:
url = "https://api-expl.odoo.com/"
db = "-api-expl-main-17145028"
username = 'admin@admin.com'
password = 'b1a41da05b8ee3c1da1484def824aa689890824a'

In [ ]:
import xmlrpc.client
common = xmlrpc.client.ServerProxy('{}/xmlrpc/2/common'.format(url))
common.version() # this is +- useless; only usefull to debug

uid = common.authenticate(db, username, password, {})

print("Check uid has a value:", uid) # Returns the user id, not usefull if you already have it

In [3]:
models = xmlrpc.client.ServerProxy('{}/xmlrpc/2/object'.format(url))

## 2 SO Creation

### 2.1: Search Partner

In [ ]:
partner = models.execute_kw(db, uid, password, 'res.partner', 'search', [[('name', '=', 'Deco Addict')]]) 
print("Partner id", partner)

In [ ]:
# Example of other queries that could have been done, with different arguments
partners = models.execute_kw(db, uid, password, 'res.partner', 'search', [[('name', 'ilike', 'Deco')]]) 
print("Partner ids (sans limit/offset)", partners)

partners = models.execute_kw(db, uid, password, 'res.partner', 'search', [[('name', 'ilike', 'Deco')]], {'limit': 1, 'offset': 1})
print("Partner ids (avec limit/offset)", partners)

### 2.2: Recherche du produit

You can see below 3 querries, they all returns exactly the same result. The only difference is where the arguments 'domain' and 'fields' are located. You can add them either in the list argument (based on their position in method signature) or the dictionnary one (based on the key)

In [ ]:
product_var = models.execute_kw(db, uid, password, 'product.product', 'search_read', [], {'domain': [('default_code', '=', 'E-COM11')], 'fields': ['id', 'name', 'default_code']}) 

print("Product variant", product_var)

In [ ]:
product_var = models.execute_kw(db, uid, password, 'product.product', 'search_read', [[('default_code', '=', 'E-COM11')]], {'fields': ['id', 'name', 'default_code']}) 

print("Product variant", product_var)

In [ ]:
product_var = models.execute_kw(db, uid, password, 'product.product', 'search_read', [[('default_code', '=', 'E-COM11')], ['id', 'name', 'default_code']]) 

print("Product variant", product_var)

### 2.3: SO Creation

In [ ]:
so_data = {
    'partner_id': partner[0],
    'order_line': [(0, 0, {'product_id': product_var[0]['id']})]
}
so = models.execute_kw(db, uid, password, 'sale.order', 'create', [so_data])

print("SO id", so)

### 2.4: Line Addition

In [ ]:
returned_value = models.execute_kw(db, uid, password, 'sale.order', 'write', [[so], {'order_line': [(0, 0, {'product_id': product_var[0]['id']})]}])

print("Returned value", returned_value)

## 3 Send by mail

Send by mail button creates a wizard, we have to deal with that: create the wizard with the necessary informations (mail templates,...) then send the mail. 

In [ ]:
# 'push' on send mail button, in user  interface this create the wizard. Here it returns a dictionnary with the infos required to create the wizard (with key 'context')
action_send_mail = models.execute_kw(db, uid, password, "sale.order", "action_quotation_send", [so])

from pprint import pprint
print("Action send mail (returned wizard)")
pprint(action_send_mail)

In [ ]:
# Wizard creation, with context from previous request
action_context = action_send_mail.get("context")

wizard_args = [{}]
wizard_kwargs = {"context": action_context}

wizard_id = models.execute_kw(db, uid, password, "mail.compose.message", "create", wizard_args, wizard_kwargs)

In [ ]:
# Send the mail
wizard_close = models.execute_kw(db, uid, password, "mail.compose.message", "action_send_mail", [wizard_id])

print("Returned value", wizard_close)

## 4: Confirm SO

In [ ]:
returned_value = models.execute_kw(db, uid, password, 'sale.order', 'action_confirm', [[so]])

print("Returned Value", returned_value)

## 5: Add log note

In [ ]:
returned_value = models.execute_kw(db, uid, password, 'sale.order', 'message_post', [[so]], {'body': 'Ce Bon de Commande a été confirmé via l\'API'})

print("Returned Value", returned_value)

# NB =================================================================

## 1) Api respects access rights

In [ ]:
so_test_access = models.execute_kw(db, uid, password, 'sale.order', 'search_read', [[('id', '=', so)]], {'fields': ['id', 'name', 'state']})

print("With admin access right we cann read the so", so_test_access)

### Creation of a nuew user, without sale acces rights

In [ ]:
username = 'new@intern.lu'
intern_password = 'password'

try:
    # creation of a new user
    new_user = models.execute_kw(db, uid, password, 'res.users', 'create', [{'name': 'New Intern', 'login': username, 'password': intern_password}])
    # Remove sales access right (only keep 'my sale only' group)
    models.execute_kw(db, uid, password, 'res.users', 'write', [[new_user], {'groups_id': [(3, 22), (3, 23)]}])

    print("New user created with id:", new_user)
except Exception as e:
    print("Error while creating new user:", e)
    new_user = models.execute_kw(db, uid, password, 'res.users', 'search', [[('login', '=', username)]])
    print("Users already exists, id:", new_user)

### Connection with this new user

In [ ]:
intern_uid = common.authenticate(db, username, intern_password, {})

print("Check uid has a value:", intern_uid)

In [ ]:
so_test_access = models.execute_kw(db, intern_uid, intern_password, 'sale.order', 'search_read', [[('id', '=', so)]], {'fields': ['id', 'name', 'state']})

print("Without admin ACR, we can't read the SO", so_test_access)

## 2) Request limit on odoo online/SH
Can be limited to 1 requests/sec. Note that on odoo SH it looks like the request are only slowed down (a lot)

In [ ]:
for i in range(1000):
    try:
        so_test_num_requests = models.execute_kw(db, uid, password, 'sale.order', 'search_read', [[('id', '=', so)]], {'fields': ['id', 'name', 'state']})
    except Exception as e:
        print("Error while requesting info, i:", i)
        print(e)
        break